In [68]:
import pandas as pd
import pymssql
import numpy as np
from numpy.linalg import norm


# import SQL database connection strings - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
from DashBoard.pages.config import database
from DashBoard.pages.config import username
from DashBoard.pages.config import password
from DashBoard.pages.config import server

from DashBoard.pages.config import Garnishes
from DashBoard.pages.config import Glassware
from DashBoard.pages.config import Ingredients
from DashBoard.pages.config import Instructions
from DashBoard.pages.config import Instructions_by_Drink
from DashBoard.pages.config import Liquors
from DashBoard.pages.config import Measured_Ingredients
from DashBoard.pages.config import Measurements
from DashBoard.pages.config import Recipes
from DashBoard.pages.config import Sources

In [69]:
try:
    conn = pymssql.connect(server,username, password,database)

    cursor = conn.cursor()
    query = f"""
    SELECT DISTINCT
        R.RecipeID,
        R.Cocktail_Name,
        L.Liquor_Name,
        Gl.Glassware_Name,
        G.Garnish_Name,
        S.Source,
        MI.Ingredients_List,
        MI.IngredientsID_List,
        MI.Ingredients,
        I.Instructions
        FROM {Recipes} AS R
    INNER JOIN {Liquors} AS L ON R.LiquorID = L.LiquorID
    INNER JOIN {Glassware} AS Gl ON R.GlasswareID = Gl.GlasswareID
    INNER JOIN {Garnishes} AS G ON R.GarnishID = G.GarnishID
    INNER JOIN {Sources} AS S ON R.SourceID = S.SourceID
    -- Subquery of Measurements concatenated Ingredients in list form 
    INNER JOIN (SELECT DISTINCT
                R.RecipeID,
                R.Cocktail_Name,
                STRING_AGG(ING.Ingredient_Name, '&,& ') AS Ingredients_List,
                STRING_AGG(ING.IngredientID, '&,& ') AS IngredientsID_List,
                STRING_AGG(CONCAT(M.Measurement_Amount,' ', ING.Ingredient_Name), '&,& ') AS Ingredients
                FROM {Recipes} AS R
            INNER JOIN {Liquors} AS L ON R.LiquorID = L.LiquorID
            INNER JOIN {Measured_Ingredients} AS MI ON R.RecipeID = MI.RecipeID
            INNER JOIN {Ingredients} AS ING ON MI.IngredientID = ING.IngredientID
            INNER JOIN {Measurements} AS M ON MI.MeasurementID = M.MeasurementID
            GROUP BY R.RecipeID, R.Cocktail_Name)
        AS MI ON R.Cocktail_Name = MI.Cocktail_Name
    -- Subquery of Instructions in list form 
    INNER JOIN (SELECT DISTINCT
                R.RecipeID,
                R.Cocktail_Name,
                STRING_AGG(I.Instruction, '&,& ') WITHIN GROUP (ORDER BY I.Instruction) AS Instructions
                FROM {Recipes} AS R
            INNER JOIN {Instructions_by_Drink} AS ID ON R.RecipeID = ID.RecipeID
            INNER JOIN {Instructions} AS I ON ID.InstructionID = I.InstructionID
            GROUP BY R.RecipeID, R.Cocktail_Name)
        AS I ON R.Cocktail_Name = I.Cocktail_Name
    ORDER BY R.RecipeID, R.Cocktail_Name ASC;
    """
    Cocktails = pd.read_sql(query, conn)
except Exception as e:
    print(e)

Cocktails["Cocktail_Name"] = Cocktails["Cocktail_Name"].astype(str)
Cocktails["Liquor_Name"] = Cocktails["Liquor_Name"].astype(str)
Cocktails["Glassware_Name"] = Cocktails["Glassware_Name"].astype(str)
Cocktails["Garnish_Name"] = Cocktails["Garnish_Name"].astype(str)
Cocktails["Source"] = Cocktails["Source"].astype(str)
Cocktails["Ingredients_List"] = Cocktails.apply(lambda x : x["Ingredients_List"].split("&,& "), axis = 1)
Cocktails["IngredientsID_List"] = Cocktails.apply(lambda x : x["IngredientsID_List"].split("&,& "), axis = 1)
Cocktails["Ingredients"] = Cocktails.apply(lambda x : x["Ingredients"].split("&,& "), axis = 1)
Cocktails["Instructions"] = Cocktails.apply(lambda x : x["Instructions"].split("&,& "), axis = 1)
Cocktails.head()

,RecipeID,Cocktail_Name,Liquor_Name,Glassware_Name,Garnish_Name,Source,Ingredients_List,IngredientsID_List,Ingredients,Instructions
0,1,#106 Cocktail,Tequila,Coupe glass,Sage,"Bartender: Chelsea Little, Bar: Olive & Oak, L...","[Chinaco Blanco Tequila, Luxardo Bitter Bianco...","[90, 198, 845, 184]","[1 oz Chinaco Blanco Tequila, 1 oz Luxardo Bit...","[1: Shake And Double Strain Into A Coupe., 2: ..."
1,2,10 to 7,Gin,Martini glass,Lemon Peel Rind,"Bartender: Gabriele Modica, Bar: Farina, Locat...","[St. Germain, Luxardo Maraschino Liqueur, No. ...","[614, 202, 229, 184]","[1 oz St. Germain, 1 oz Luxardo Maraschino Liq...",[1: Shake And Serve Up In A Martini Glass With...
2,3,1626,Whiskey,Cocktail Glass,Unknown,Unknown,"[Angostura Bitters, Italian Preserved Cherry, ...","[354, 1141, 66, 479, 87]","[2 oz Angostura Bitters, 1 oz Italian Preserve...",[1: Shake well with ice and strain into chille...
3,4,1880's Americana,Bourbon,Old Fashioned Glass,Cola candied orange peel,Unknown,"[Amer Picon, House Cola Syrup, Hirsch Small Ba...","[351, 799, 165]","[10 ml Amer Picon, 10 ml House Cola Syrup, 2 o...",[1: Build Over Ice In Old Fashioned Glass.]
4,5,19th Century,Whiskey,Cocktail Glass,Unknown,Unknown,"[Fresh Lemon Juice, Bourbon Whiskey, Lillet Ro...","[460, 66, 517, 649]","[0.75 oz Fresh Lemon Juice, 1.5 oz Bourbon Whi...",[1: Shake with ice and strain into chilled coc...


In [70]:
try:
    conn = pymssql.connect(server,username, password,database)

    cursor = conn.cursor()
    query_1 = f"""
    SELECT DISTINCT
        I.IngredientID,
        I.Ingredient_Name
        FROM {Ingredients} AS I
    ORDER BY I.Ingredient_Name ASC;
    """
    Ingredients_Distinct = pd.read_sql(query_1, conn)
except Exception as e:
    print(e)

Ingredients_Distinct.head(20)

,IngredientID,Ingredient_Name
0,1074,- 3 Drops Hot Red Pepper Sauce
1,1,100- Or 101-Proof Rye
2,2,100-Proof Vodka
3,346,15 Yo Sherry
4,3,151-Proof Rum
5,4,17-Year-Old J. Wray And Nephew Ltd. Rum
6,5,3/4 Ozanejo Tequila
7,6,3/4 Ozblended Scotch Whiskey
8,347,7-Spice Syrup
9,1075,750-Ml Cognac


In [71]:
Cocktails_limited = Cocktails[["RecipeID", "Cocktail_Name", "Liquor_Name", "Ingredients_List"]]
Cocktails_limited.head()

,RecipeID,Cocktail_Name,Liquor_Name,Ingredients_List
0,1,#106 Cocktail,Tequila,"[Chinaco Blanco Tequila, Luxardo Bitter Bianco..."
1,2,10 to 7,Gin,"[St. Germain, Luxardo Maraschino Liqueur, No. ..."
2,3,1626,Whiskey,"[Angostura Bitters, Italian Preserved Cherry, ..."
3,4,1880's Americana,Bourbon,"[Amer Picon, House Cola Syrup, Hirsch Small Ba..."
4,5,19th Century,Whiskey,"[Fresh Lemon Juice, Bourbon Whiskey, Lillet Ro..."


In [72]:
def list_breakdown(list_):
    result = []
    for i in list_:
        i = i.strip()
        if(i == ""):continue
        else: result += i.split(" ")
    return result

In [80]:
# Ingredients to cocktails recommendations 

def Similarity_Added_labels(df_drinks, drink_ingredients_list):

    def Similarity_Measure(other_ingredient_list, drink_ingredient_list):
        drink_1 = list_breakdown(list(map(str, other_ingredient_list)))
        drink_2 = list_breakdown(list(map(str, drink_ingredient_list)))
        measure = 0
        for num in drink_2:
            if(num.title() in drink_1):
                measure += 1 
        return measure

    df_drinks["Similarity"] = df_drinks.apply(lambda x : Similarity_Measure(x["Ingredients_List"], drink_ingredients_list), axis = 1)
    df_drinks = df_drinks.sort_values(by = "Similarity", ascending = False)
    max_measure = df_drinks["Similarity"].max()
    df_drinks_sim = df_drinks[df_drinks["Similarity"] == max_measure]
    # extra constaints based on resulting similarities df
    if(max_measure == 0):
        return None
    else:
        similarity_count_df = pd.DataFrame(df_drinks["Similarity"].value_counts()).reset_index().sort_values(by="index", ascending= False)
        num_sim_cocktails = similarity_count_df.head(1)["Similarity"].tolist()[0]
        if(num_sim_cocktails == 3):
            return df_drinks_sim
        else:
            if(num_sim_cocktails < 3):
                # Finds 3 drinks based on similarity measure
                other_drinks = df_drinks.sample(0)
                for i in range(1,max_measure+1):
                    other_drinks = pd.concat([other_drinks, df_drinks[df_drinks["Similarity"] == (max_measure - i)]], axis=0)
                    if(len(other_drinks) < 3): continue
                    else: break
                # Returns 3 drinks in order of similarity measure
                result_df_ranked = df_drinks_sim
                for i in range(1,max_measure+1):
                    if(len(result_df_ranked) == 3):break
                    elif(len(result_df_ranked) < 3):
                        if(len(other_drinks[other_drinks["Similarity"] == (max_measure - i)]) >= 1):
                            if(len(other_drinks[other_drinks["Similarity"] == (max_measure - i)]) == 1):
                                result_df_ranked = pd.concat([result_df_ranked, other_drinks[other_drinks["Similarity"] == (max_measure - i)].sample(1)], axis=0)
                            else:
                                result_df_ranked = pd.concat([result_df_ranked, other_drinks[other_drinks["Similarity"] == (max_measure - i)].sample(3-len(result_df_ranked))], axis=0)
                        else:
                            # this is when the other_drinks has length 0, meaning we can bypass it sense it has no information
                            continue
                    
                return result_df_ranked
            else:
                return df_drinks_sim.sample(3)


In [83]:
liquor_choosen = "cherry"
choosen_1 = "k"

choosen_2 = "c"

drink_test = ["cherry", "lime", "vodka", "juice"]
Cocktails_limited_1 = Cocktails.copy()
test = [liquor_choosen] # weird error
# drink_IDs = ID_Ingredient_Transformer(drink_test, Ingredients_Distinct)
Cocktails_limited_1 = Similarity_Added_labels(Cocktails_limited_1, drink_test)
#Cocktails_limited_1 = Cocktails_limited_1.sort_values(by="Similarity", ascending= False)
Cocktails_limited_1

,RecipeID,Cocktail_Name,Liquor_Name,Glassware_Name,Garnish_Name,Source,Ingredients_List,IngredientsID_List,Ingredients,Instructions,Similarity
272,273,Cherry Wine Cocktail,Wine,Cocktail Glass,Unknown,Unknown,"[Vodka, Danish Cherry Wine, Juice Of A Lime]","[336, 115, 176]","[0.75 oz Vodka, 0.75 oz Danish Cherry Wine, 0....",[1: Shake with ice and strain into cocktail gl...,4
1023,1025,Rococo,Vodka,Cocktail Glass,Unknown,Unknown,"[Cherry Vodka, Orange Juice, Triple Sec]","[86, 261, 328]","[1 oz Cherry Vodka, 1 oz Orange Juice, 0.5 oz ...",[1: Shake with ice and strain into cocktail gl...,3
1323,1319,Twister,Vodka,Collins Glass,Unknown,Unknown,"[Fresh Lime Juice, Vodka]","[463, 336]","[0.5 oz Fresh Lime Juice, 2 oz Vodka]",[1: Pour vodka and lime juice into Collins gla...,3


In [84]:
# Cocktail to cocktails recommendations 

def Similarity_Added_labels_Cocktail(df_drinks, drink_ingredients_list, cocktail_names):

    def Similarity_Measure(other_ingredient_list, drink_ingredient_list):
        drink_1 = list_breakdown(list(map(str, other_ingredient_list)))
        drink_2 = list_breakdown(list(map(str, drink_ingredient_list)))
        measure = 0
        for num in drink_2:
            if(num.title() in drink_1):
                measure += 1 
        return measure
    for cocktail in cocktail_names:
        df_drinks = df_drinks[df_drinks["Cocktail_Name"] != f"{cocktail}"]
    df_drinks["Similarity"] = df_drinks.apply(lambda x : Similarity_Measure(x["Ingredients_List"], drink_ingredients_list), axis = 1)
    df_drinks = df_drinks.sort_values(by = "Similarity", ascending = False)
    max_measure = df_drinks["Similarity"].max()
    df_drinks_sim = df_drinks[df_drinks["Similarity"] == max_measure]
    # extra constaints based on resulting similarities df
    if(max_measure == 0):
        return None
    else:
        similarity_count_df = pd.DataFrame(df_drinks["Similarity"].value_counts()).reset_index().sort_values(by="index", ascending= False)
        num_sim_cocktails = similarity_count_df.head(1)["Similarity"].tolist()[0]
        if(num_sim_cocktails == 3):
            return df_drinks_sim
        else:
            if(num_sim_cocktails < 3):
                # Finds 3 drinks based on similarity measure
                other_drinks = df_drinks.sample(0)
                for i in range(1,max_measure+1):
                    other_drinks = pd.concat([other_drinks, df_drinks[df_drinks["Similarity"] == (max_measure - i)]], axis=0)
                    if(len(other_drinks) < 3): continue
                    else: break
                # Returns 3 drinks in order of similarity measure
                result_df_ranked = df_drinks_sim
                for i in range(1,max_measure+1):
                    if(len(result_df_ranked) == 3):break
                    elif(len(result_df_ranked) < 3):
                        if(len(other_drinks[other_drinks["Similarity"] == (max_measure - i)]) >= 1):
                            if(len(other_drinks[other_drinks["Similarity"] == (max_measure - i)]) == 1):
                                result_df_ranked = pd.concat([result_df_ranked, other_drinks[other_drinks["Similarity"] == (max_measure - i)].sample(1)], axis=0)
                            else:
                                result_df_ranked = pd.concat([result_df_ranked, other_drinks[other_drinks["Similarity"] == (max_measure - i)].sample(3-len(result_df_ranked))], axis=0)
                        else:
                            # this is when the other_drinks has length 0, meaning we can bypass it sense it has no information
                            continue
                    
                return result_df_ranked
            else:
                return df_drinks_sim.sample(3)

In [86]:
ing_test = Cocktails[Cocktails["Cocktail_Name"] == "Classic Martini"]["Ingredients_List"].tolist()[0]
Cocktails_limited_1 = Cocktails.copy()
Cocktails_limited_1 = Similarity_Added_labels_Cocktail(Cocktails_limited_1, ing_test, ["Classic Martini"])
#Cocktails_limited_1 = Cocktails_limited_1.sort_values(by="Similarity", ascending= False)
Cocktails_limited_1

C:\Users\Adam Brewer\AppData\Local\Temp\ipykernel_3548\3202612831.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drinks["Similarity"] = df_drinks.apply(lambda x : Similarity_Measure(x["Ingredients_List"], drink_ingredients_list), axis = 1)


,RecipeID,Cocktail_Name,Liquor_Name,Glassware_Name,Garnish_Name,Source,Ingredients_List,IngredientsID_List,Ingredients,Instructions,Similarity
1076,1078,Sawyer,Gin,Double Rocks Glass,Unknown,Bitters: A Spirited History of a Classic Cure-all,"[Peychaud'S Bitters, Orange Bitters, Bitters #...","[847, 260, 49, 354, 602, 469, 147]","[7 dashes Peychaud'S Bitters, 7 dashes Orange ...",[1: Combine all the ingredients in a cocktail ...,4
955,957,Plain Vermouth Cocktail,Other,Cocktail Glass,Unknown,Unknown,"[Orange Bitters, Sweet Vermouth, Dry Vermouth]","[260, 320, 123]","[1 oz Orange Bitters, 1 oz Sweet Vermouth, 1 o...",[1: See Vermouth Cocktail on page 189.],3
862,864,Nikka's Golden Cherry,Whisky,Cocktail Glass,Unknown,"Bartender: Torrence Swain, Location: Washingto...","[Luxardo Maraschino Liqueur, Nikka Coffey Grai...","[202, 223, 752, 912]","[0.5 oz Luxardo Maraschino Liqueur, 2 oz Nikka...","[1: Stir Ingredients Over Cracked Ice., 2: Wit...",3
